[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mfurio93/Used-Cars-Dataset/blob/main/Notebook.ipynb)

In [41]:
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# File importation and cleaning

We now import the .csv file obtained from https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data.

It contains all relevant information that Craigslist provides on car sales including columns like price, cylinders, manufacturer, latitude, and 21 other categories.

Pandas informs us that we're working with a pretty large dataset of over 400000 entries.

Note: due to the massive size of the file (over 1 GB), the next code block might take around 2 minutes to process.

In [42]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = '1Pt1akrURtchh4elqaWo4n_bskLI4nJme'
download = drive.CreateFile({'id':file_id})
download.GetContentFile('file.csv')

import pandas as pd
df = pd.read_csv("file.csv")
pd.set_option('display.float_format', lambda x: '%.0f' % x)
df.shape

(426880, 26)

# Analysis of missing data

Through the use of Pandas, we can see that there are columns with way too much missing data, so we'll delete those columns entirely. Others, such as latitude or longitude, are likely not useful for our models.

Additionally, the columns where there is a more despreciable amount of missing data will be kept, however, the rows with the missing data of these columns will also be deleted.

Last but not least, we delete all duplicate entries within the dataframe.

In [43]:
df.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year              1205
manufacturer     17646
model             5277
condition       174104
cylinders       177678
fuel              3013
odometer          4400
title_status      8242
transmission      2556
VIN             161042
drive           130567
size            306361
type             92858
paint_color     130203
image_url           68
description         70
county          426880
state                0
lat               6549
long              6549
posting_date        68
dtype: int64

In [44]:
#Likely not useful columns
del df['id']
del df['url']
del df['region_url']
del df['VIN']
del df['image_url']
del df['description']
del df['county']
del df['lat']
del df['long']
del df['posting_date']

In [45]:
#Columns with too much missing data
del df['condition']
del df['cylinders']
del df['drive']
del df['size']
del df['paint_color']
del df['type']

In [46]:
#Rows with too much missing data
df.fillna(0, inplace=True)
df.drop(df[(df['price'] == 0)].index, inplace=True)
df.drop(df[(df['year'] == 0)].index, inplace=True)
df.drop(df[(df['manufacturer'] == 0)].index, inplace=True)
df.drop(df[(df['model'] == 0)].index, inplace=True)
df.drop(df[(df['fuel'] == 0)].index, inplace=True)
df.drop(df[(df['odometer'] == 0)].index, inplace=True)
df.drop(df[(df['title_status'] == 0)].index, inplace=True)
df.drop(df[(df['transmission'] == 0)].index, inplace=True)

In [47]:
#Dropping duplicates
df.drop_duplicates(inplace=True)

# Analysis of target variable

Now we take a look at how our target variable behaves.

Here we can appreciate several values that seem either off or excessive. To curb this effect, we'll delete all rows whose target variable value is outside of the \$1000 (one thousand) to \$100000 (one hundred thousand).

After this cleaning process, we lost roughly 120000 entries, or about 30% of our dataset, however, we now have a completely filled and unique dataset.

In [48]:
df.price.describe()

count       315296
mean         76518
std       12726399
min              1
25%           7500
50%          15990
75%          27990
max     3736928711
Name: price, dtype: float64

In [49]:
df.drop(df[df['price'] > 100000].index, inplace=True)
df.drop(df[df['price'] < 1000].index, inplace=True)
df.price.describe()

count   307166
mean     19492
std      14165
min       1000
25%       7995
50%      16495
75%      28223
max     100000
Name: price, dtype: float64

# Model Creation

...